In [2]:
from extract_summary_finetune.extract_summary_from_news import ExtratSummary

In [4]:
lstm_summary = ExtratSummary(network="lstm1",bert="swufe_bert",loss="BCEloss")

In [5]:
lstm_summary.summary[0]

'[汽车之车官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万续航版的ND里6'

In [6]:
news_content_filled=lstm_summary.news_content_filled

In [17]:
import re

def divide_sentences(news_content_filled:list):
    news_to_sentences=[]
    for content in news_content_filled:
        news_content = re.split(r'(\!|\?|。|！|？|\.{6}|\,|，)', content)
        news_to_sentences.append(news_content)
    return news_to_sentences

In [21]:
a = divide_sentences(news_content_filled)[0]

In [16]:
sum([len(item) for item in a])


512

In [22]:
import numpy as np
def modify_labels(predict_label,news_content_filled):
    news_to_sentences = divide_sentences(news_content_filled)
    predict_label_modified = []
    for k in range(len(news_to_sentences)):
        sentences_length_list = [len(sentence) for sentence in news_to_sentences[k]]
        start_index = 0
        label_modified_list = []
        for i in range(len(sentences_length_list)):
            end_index = start_index + sentences_length_list[i]
            label_divided = list(predict_label[k][start_index:end_index])
            if sum(label_divided) > 0.5*len(label_divided):
                label_modified = [True for item in label_divided]
            elif sum(label_divided) < 0.2*len(label_divided):
                label_modified = [False for item in label_divided]
            else:
                label_modified = label_divided
            label_modified_list.extend(label_modified)
            start_index = end_index
        predict_label_modified.append(np.array(label_modified_list))
    return np.array(predict_label_modified)
            

In [26]:
pred = lstm_summary.predict_label
w = modify_labels(pred,news_content_filled)


In [30]:
np.sum(w == pred)
lstm_summary.

25153

In [33]:
def modify_summary(modify_labels):
        word = lstm_summary.news_to_words()
        summary = []
        for i in range(modify_labels.shape[0]):
            each_summary = ''
            for j in range(modify_labels.shape[1]):
                each_summary = each_summary + (modify_labels[i][j]*word[i][j])
            summary.append(each_summary)
        return summary
 


In [36]:
modify_summary(w)[0:5]

/tmp/ipykernel_935730/1810000260.py:7: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  each_summary = each_summary + (modify_labels[i][j]*word[i][j])


['[汽车之车我们从官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万元续航版的ND里6',
 '揭秘Siri，苹果发布论文阐释语音助手设计想法就多任务处理、多语言识别等问题，苹果在论文中给出了自己不同的想法。在第一篇论文中，苹果就语音助手中的多任务处理问题进行了阐释，它指出在Siri中，唤醒处理通常需要两个步骤：AI首先必须确定输入容内',
 '在2018年的大牛行情过后，2019年的债市将如何演绎？多位基金经理在接受记者采访时表示，去年的快速上涨收窄了债市继续上扬的空间，尤其是利率债的机会明显减少，但整体牛市的格局不会变化，今年可能处于阶段性的“牛尾行情”。基于这一判断许多基金经理将债市淘金的对象指向信用债，期望从“信用利差”中获取超额收益',
 '记者 ，总投资921亿元的广西“信息网”基础设施建设三年大会战正式启动。据了解，今年2月份，广西发布了《“信息网”基础设施建设三年大会战实施方案》，明确全区3年计划投资921亿元，建设282个项目。4月12日，共5个“信息网”新型基础设施项目集中开工',
 '从某种程度上来看，这是因为腾讯是一个强大的生态企业，而且有着诸多的现金奶牛业务，如我们熟悉的文娱、游戏、音乐、内容等生态，这些业务的营收太好，以至于使得腾讯广告或者说腾讯在流量变现上相对求稳。腾讯顺应大趋势，积极赋能微盟等SaaS服务商，帮助这些服务商推出更多垂直领域的SaaS解决方案。']

In [37]:
modify_summary(pred)[0:5]

/tmp/ipykernel_935730/1810000260.py:7: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  each_summary = each_summary + (modify_labels[i][j]*word[i][j])


['[汽车之车官方获悉，特斯拉（中国）官网针对在售的Model S(参数|询价)长续航版、Performance高性能版的价格进行了下调，调整后售价为73.39万、83.39万元，降价幅度均为2.3万续航版的ND里6',
 '揭秘Siri，苹果发布论文阐释语音助手设计想法就多任务处理、多语言识别等问题，苹果在论文中给出了自己不同的想法。在第一论文中，苹果就语音助手中的多任务处理问题进行了阐释，它指出在Siri中，唤醒处理通常需要两个步骤：AI首先必须确定输入容内',
 '在2018年的大牛行情过后，2019年的债市将如何演绎？多位基金经理在接受记者采访时表示，去年的快速上涨收窄了债市继续上扬的空间，尤其是利率债的机会明显减少，但整体牛市的格局不会变化，今年可能处于阶段性的“牛尾行情”。基这判断多基金经理将债市淘金的对象指向信，期望从“信用利差”中获超额收紧“信用利',
 '记者 ，总投资921亿元的广西“信息网”基础设施建设三年大会战正式启动。据了解，今年2月份，广西发布了《“信息网”基础设施建设三年大会战实施方案》，明确全区3年计划投资921亿元，建设282个项目。4月12日，共5个“信息网”新型基础设施项目集',
 '从某种程度上来看，这是因为腾讯是一个强大的生态企业，而且有着诸多的现金奶牛业务，如我们熟悉的文娱、游戏、音乐、内容等生态，这些业务的营收太好，以至于使得腾讯广告或者说腾讯在流量变现上相对求稳。腾讯顺应大趋势，积极赋能微盟等SaaS服务商，帮助这些服务商推出更多垂直领域的SaaS解决方案。微盟将']

In [10]:
import re
from zhon import hanzi


def divide_sentences2(news_content_filled:list):
    news_divide_sentences=[]
    for content in news_content_filled:
        news_content = re.findall(hanzi.sentence, content)
        news_divide_sentences.append(news_content)
    return news_divide_sentences





In [12]:
a = divide_sentences2(news_content_filled)
a

[['万元。', '秒。', '版，此次调价仅针对前面两款车型。', '万元。', '万元。', '。', '）。'],
 ['，苹果发布论文阐释语音助手设计想法就多任务处理、多语言识别等问题，苹果在论文中给出了自己不同的想法。',
  '。',
  '小时的音频带有语音标签，其余均只有扬声器标签。',
  '相比于一般训练模型去获取多个标签的思路，苹果通过将不同任务的训练数据进行级联来训练多个相关任务的模型。',
  '在另一篇论文中，苹果还介绍了多语言演讲场景的演讲者识别系统设计——知识图谱辅助听写系统决策。',
  '以声学子模型为例，它可以基于语音信号传输痕迹来进行预测，并且其上下文感知的预测组件考虑了各种交互上下文信号，其中上下文信号包含有关发出命令的条件信息、已安装的命令语言环境、当前选择的命令语言环境以及用户在发出请求之前是否切换命令语言环境的信息。'],
 ['年的债市将如何演绎？',
  '多位基金经理在接受记者采访时表示，去年的快速上涨收窄了债市继续上扬的空间，尤其是利率债的机会明显减少，但整体牛市的格局不会变化，今年可能处于阶段性的“牛尾行情”。',
  '基于这一判断，许多基金经理将债市淘金的对象指向信用债，期望从“信用利差”中获取超额收益。',
  '在基金经理眼里，利率债的强势上扬催生了去年的大牛市，今年信用债有可能成为债市新主角，为债基提供了结构性投资机会。',
  '她指出，历史上的债券行情，一般以利率债为开端，此后轮动到高、中、低等级信用债，最后以垃圾债收尾。',
  '年四季度才有所好转。',
  '随着今年利率债的投资价值降低和宽货币效果的逐步显现，利率债和信用债的利差有望进一步压缩，信用债的票息优势会逐步体现，叠加其收益率的下行，信用债将成为今年最大的看点。',
  '股市场开春以来的大幅上涨，也让可转债成为债基眼中的“香饽饽”。'],
 ['亿元的广西“信息网”基础设施建设三年大会战正式启动。',
  '个项目。',
  '亿元。',
  '年底竣工并投入运营，项目建成后将为广西、西南及东盟的客户提供政务云、金融云、农业云、产业云等服务及同城、异地灾备服务。',
  '精品网。',
  '连续覆盖。'],
 ['从某种程度上来看，这是因为腾讯是一个强大的生态企业，而且有着诸多的现金奶牛业务，如我们熟悉的文娱、游戏、音乐、内容